# Decomposição LU

In [180]:
from typing import Callable
import numpy as np
import matplotlib.pyplot as plt

# Decomposição LU
Algoritmos para fazer uma decomposição LU.

## Fatorização de Crout

In [181]:
def lucrout(A: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """
        Dado uma matriz `A` calcula a sua decomposição LU sem pivotagem usando fatorização de Crout.

        ### Argumentos
        A: Matriz N por N

        ### Retorno
        L: Matriz N por N lower
        U: Matriz N por N upper
    """

    # Dimensões em jogo
    N = A.shape[0]
    L, U = np.zeros((N, N)), np.identity(N)
    
    # Executar a decomposição LU geral
    for i in range(N):
        L[i:, i] = A[i:, i] - L[i:, :i] @ U[:i, i]
        U[i, i+1:] = (A[i, i+1:] - L[i, :i] @ U[:i, i+1:]) / L[i, i]
    
    return L, U


# Versão mais legível:
'''
N = A.shape[0]
L, U = np.zeros((N, N)), np.identity(N)

for i in range(N):
    for j in range(i, N):
        L[j, i] = A[j, i] - L[j, :j] @ U[:j, i]
    
    div = L[i][i]
    for j in range(i + 1, N):
        U[i, j] = (A[i, j] - L[i, :i] @ U[:i, j]) / div

return L, U
'''

'\nN = A.shape[0]\nL, U = np.zeros((N, N)), np.identity(N)\n\nfor i in range(N):\n    for j in range(i, N):\n        L[j, i] = A[j, i] - L[j, :j] @ U[:j, i]\n    \n    div = L[i][i]\n    for j in range(i + 1, N):\n        U[i, j] = (A[i, j] - L[i, :i] @ U[:i, j]) / div\n\nreturn L, U\n'

## Fatorização de Doolittle

In [182]:
def ludoolittle(A: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """
        Dado uma matriz `A` calcula a sua decomposição LU sem pivotagem usando fatorização de Doolittle.

        ### Argumentos
        A: Matriz N por N

        ### Retorno
        L: Matriz N por N lower
        U: Matriz N por N upper
    """

    # Dimensões em jogo
    N = A.shape[0]
    L, U = np.identity(N), np.zeros((N, N))
    
    # Executar a decomposição LU geral
    for i in range(N):
        U[i, i:] = A[i, i:] - L[i, :i] @ U[:i, i:]
        L[i+1:, i] = (A[i+1:, i] - L[i+1:, :i+1] @ U[:i+1, i]) / U[i, i]
    
    return L, U


# Versão mais legível:
'''
# Dimensões em jogo
    N = A.shape[0]
    L, U = np.identity(N), np.zeros((N, N))

# Executar a decomposição LU geral
for i in range(N):
    for j in range(i, N):
        U[i, j] = A[i, j] - L[i, :j] @ U[:j, j]
    
    div = U[i][i]
    for j in range(i + 1, N):
        L[j, i] = (A[j, i] - L[j, :j] @ U[:j, i]) / div

return L, U
'''

'\n# Dimensões em jogo\n    N = A.shape[0]\n    L, U = np.identity(N), np.zeros((N, N))\n\n# Executar a decomposição LU geral\nfor i in range(N):\n    for j in range(i, N):\n        U[i, j] = A[i, j] - L[i, :j] @ U[:j, j]\n    \n    div = U[i][i]\n    for j in range(i + 1, N):\n        L[j, i] = (A[j, i] - L[j, :j] @ U[:j, i]) / div\n\nreturn L, U\n'

## Eliminação Gaussiana

In [183]:
def lugauss(Ao: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """
        Dado uma matriz `Ao` calcula a sua decomposição LU sem pivotagem usando eliminação gaussiana.

        ### Argumentos
        Ao: Matriz N por N

        ### Retorno
        L: Matriz N por N lower
        U: Matriz N por N upper
    """

    # Dimensões em jogo
    N = Ao.shape[0]
    L = np.zeros((N, N))

    # Evitar side effects
    A = np.copy(Ao)

    # A matriz U eventualmente será uma matriz upper!
    U = A

    for i in range(N):
        # Criar a matriz L_i
        L_i = np.identity(N)
        L_i[i+1:, i] = -U[i+1:, i]
        L_i[i:, i] /= U[i, i]

        # Guardar a coluna i de U em L
        L[i:, i] = U[i:, i]

        # Aplicar L_i a A
        U = L_i @ U
    
    return L, U

# Resolver Sistemas de Equações Lineares

Podemos usar a decomposição LU para resolver sistemas de equações lineares quaisquer!

## Método Geral

In [184]:
def lusolve(Ao: np.ndarray, bbo: np.ndarray, ludecom: Callable) -> np.ndarray:
    """
        Resolve os M sistemas de N equações definidos por A * xx = bb usando a decomposição LU obtida pela função `ludecom(A)`.

        ### Argumentos
        Ao: Matriz N por N
        bbo: Matriz N por M
        
        ### Retorno
        xx: Matriz N por M
    """
    
    # Dimensões em jogo
    N = Ao.shape[0]

    # Evitar side effects
    A = np.copy(Ao)
    bb = np.copy(bbo)

    # Obter a decomposição LU
    L, U = ludecom(A)

    # Substituição progressiva -> Resolver L * yy = bb
    yy = np.zeros_like(bb)
    yy[0] = bb[0] / L[0, 0]
    for i in range(1, N):
        yy[i] = (bb[i] - (L[i, :i] @ yy[:i])) / L[i, i]
    
    # Substituição regressiva -> Resolver U * xx = yy
    xx = np.zeros_like(bb)
    xx[-1] = yy[-1] / U[N-1, N-1]
    for i in range(N-1, -1, -1):
        xx[i] = (yy[i] - (U[i][i+1:] @ xx[i+1:])) / U[i, i]
    
    return xx

## Eliminação Gaussiana com Pivotagem Parcial

Em rigor, a função de decomposição LU abaixo não devolve a decomposição LU de $A$, mas sim a decomposição LU de $PA$, em que $P$ é a matriz associada às permutações realizadas! Isto é suficiente para resolver sistemas de equações lineares, mas não ficamos a saber a decomposição LU de $A$.

In [185]:
def troca_linhas(A: np.ndarray, mm: np.ndarray) -> np.ndarray:
    """
        Troca as linhas de A de acordo com o mapa de endereços mm.
    """

    B = np.zeros_like(A)
    for line in range(B.shape[0]):
        B[line] = np.copy(A[mm[line]])
    return B


def lugausspp(Ao: np.ndarray) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
        Dado uma matriz A calcula a sua decomposição LU usando eliminação de Gauss com pivotagem parcial.

        ### Argumentos
        Ao: Matriz N por N
        
        ### Retorno
        L: Matriz N por N lower
        U: Matriz N por N upper
        mm: Mapa de endereços associado à troca de linhas.
    """

    # Evitar side effects
    A = np.copy(Ao)

    N = A.shape[0]

    # Fazer pivotagem usando um mapa de endereços
    mm = np.arange(0, N)
    for line in range(N):
        # Posição do maior elemento da coluna (tendo em conta as trocas já feitas)
        coluna = A[mm[line:], line]
        index = list(coluna).index(max(coluna)) + line

        # Trocar os elementos
        mm[line], mm[index] = mm[index], mm[line]
    

    # Abandonar o mapa de endereços porque se não o produto interno L @ U não funciona
    # Isto pois a matriz U não tem as linhas trocadas, teriamos de fazer uma função de produto interno que tivesse em conta o mapa de endereços
    B = troca_linhas(A, mm)

    L = np.zeros((N, N))
    U = B

    for i in range(N):
        # Criar a matriz L_i
        L_i = np.identity(N)
        L_i[i+1:, i] = -U[i+1:, i]
        L_i[i:, i] /= U[i, i]

        # Guardar a coluna i de U em L
        L[i:, i] = U[i:, i]

        # Aplicar L_i a A
        U = L_i @ U
    
    return L, U, mm


def lusolvepp(Ao: np.ndarray, bbo: np.ndarray) -> np.ndarray:
    """
        Resolve o sistema de D equações definido por A * xx = bb usando decomposição LU com pivotagem parcial.

        ### Retorno
        Devolve um array que contém o vetor solução.
    """

    # Evitar side effects
    A = np.copy(Ao)
    bb = np.copy(bbo)

    N = len(bb)

    # Obter a decomposição LU
    L, U, mm = lugausspp(A)
    bb = troca_linhas(bb, mm)

    # Substituição progressiva -> Resolver L * yy = bb
    yy = np.zeros_like(bb)
    yy[0] = bb[0] / L[0, 0]
    for i in range(1, N):
        yy[i] = (bb[i] - (L[i, :i] @ yy[:i])) / L[i][i]
    
    # Substituição regressiva -> Resolver U * xx = yy
    xx = np.zeros_like(bb)
    xx[-1] = yy[-1] / U[N-1, N-1]
    for i in range(N-1, -1, -1):
        xx[i] = (yy[i] - (U[i][i+1:] @ xx[i+1:])) / U[i][i]
    
    return xx

# Testar

Vamos fazer uma bateria de testes a este métodos!

In [186]:
from testes import testelu, testelinear, testeinv

## Decomposição LU

In [187]:
print("Decomposição LU com Fatorização de Crout\n")
testelu(lucrout, 4)

Decomposição LU com Fatorização de Crout


Matriz aleatória
[[7 0 6 5]
 [1 7 0 7]
 [9 1 1 5]
 [9 7 5 0]]

Matriz L @ U
[[ 7.00000000e+00  0.00000000e+00  6.00000000e+00  5.00000000e+00]
 [ 1.00000000e+00  7.00000000e+00 -2.77555756e-17  7.00000000e+00]
 [ 9.00000000e+00  1.00000000e+00  1.00000000e+00  5.00000000e+00]
 [ 9.00000000e+00  7.00000000e+00  5.00000000e+00  0.00000000e+00]]

Matriz L
[[  7.           0.           0.           0.        ]
 [  1.           7.           0.           0.        ]
 [  9.           1.          -6.59183673   0.        ]
 [  9.           7.          -1.85714286 -12.05882353]]

Matriz U
[[ 1.          0.          0.85714286  0.71428571]
 [ 0.          1.         -0.12244898  0.89795918]
 [ 0.          0.          1.          0.35294118]
 [ 0.          0.          0.          1.        ]]


In [188]:
print("Decomposição LU com Fatorização de Doolittle\n")
testelu(ludoolittle, 4)

# Nesta fatorização podemos ver que é normal haver erros de divisão por 0.

Decomposição LU com Fatorização de Doolittle


Matriz aleatória
[[5 4 4 6]
 [6 7 3 7]
 [2 6 5 8]
 [3 1 9 8]]

Matriz L @ U
[[5. 4. 4. 6.]
 [6. 7. 3. 7.]
 [2. 6. 5. 8.]
 [3. 1. 9. 8.]]

Matriz L
[[ 1.          0.          0.          0.        ]
 [ 1.2         1.          0.          0.        ]
 [ 0.4         2.          1.          0.        ]
 [ 0.6        -0.63636364  0.77922078  1.        ]]

Matriz U
[[ 5.         4.         4.         6.       ]
 [ 0.         2.2       -1.8       -0.2      ]
 [ 0.         0.         7.         6.       ]
 [ 0.         0.         0.        -0.4025974]]


In [189]:
print("Decomposição LU com Eliminação de Gauss\n")
testelu(lugauss, 4)

Decomposição LU com Eliminação de Gauss


Matriz aleatória
[[5 3 7 0]
 [4 3 2 4]
 [6 0 9 6]
 [1 5 9 1]]

Matriz L @ U
[[5.0000000e+00 3.0000000e+00 7.0000000e+00 0.0000000e+00]
 [4.0000000e+00 3.0000000e+00 2.0000000e+00 4.0000000e+00]
 [6.0000000e+00 8.8817842e-16 9.0000000e+00 6.0000000e+00]
 [1.0000000e+00 5.0000000e+00 9.0000000e+00 1.0000000e+00]]

Matriz L
[[  5.           0.           0.           0.        ]
 [  4.           0.6          0.           0.        ]
 [  6.          -3.6        -21.           0.        ]
 [  1.           4.4         34.          20.23809524]]

Matriz U
[[ 1.          0.6         1.4         0.        ]
 [ 0.          1.         -6.          6.66666667]
 [ 0.          0.          1.         -1.42857143]
 [ 0.          0.          0.          1.        ]]


## Resolver Equações Lineares

In [190]:
print("Decomposição com Fatorização de Crout\n")
testelinear(lambda Ao, bbo: lusolve(Ao, bbo, lucrout), 10)

Decomposição com Fatorização de Crout

Para um sistema aleatório este método difere do resultado do numpy no máximo 1.021e-14.

Para cinco sistemas aleatórios este método difere do resultado do numpy no máximo 3.659e-13.



In [191]:
print("Decomposição com Fatorização de Doolittle\n")
testelinear(lambda Ao, bbo: lusolve(Ao, bbo, ludoolittle), 10)

Decomposição com Fatorização de Doolittle

Para um sistema aleatório este método difere do resultado do numpy no máximo 1.705e-12.

Para cinco sistemas aleatórios este método difere do resultado do numpy no máximo 1.315e-13.



In [192]:
print("Decomposição com Eliminação Gaussiana\n")
testelinear(lambda Ao, bbo: lusolve(Ao, bbo, lugauss), 10)

Decomposição com Eliminação Gaussiana

Para um sistema aleatório este método difere do resultado do numpy no máximo 1.377e-14.

Para cinco sistemas aleatórios este método difere do resultado do numpy no máximo 9.770e-15.



In [193]:
print("Decomposição com Eliminação Gaussiana com Pivotagem Parcial\n")
testelinear(lusolvepp, 10)

Decomposição com Eliminação Gaussiana com Pivotagem Parcial

Para um sistema aleatório este método difere do resultado do numpy no máximo 2.576e-14.

Para cinco sistemas aleatórios este método difere do resultado do numpy no máximo 6.395e-14.

